# Grafico de Produção em sacas

## Funções

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import logging
from functools import reduce
import textwrap
import json
from datetime import datetime

# Get the current date
current_date = datetime.now()


# Configurando o logging para exibir mensagens de debug
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def _format_comment_with_breaks(text, line_length=77):
    """
    Formata uma string de comentário com quebras de linha HTML
    para largura consistente no tooltip.
    """
    if not isinstance(text, str):
        return "Nenhum comentário disponível."
    # textwrap.wrap retorna uma lista de linhas, unidas pela tag <br>
    return '<br>'.join(textwrap.wrap(text, width=line_length))

In [ ]:
# --- Funções de Processamento de Dados (Parte 1) ---
def _ler_e_processar_modelo_csv(arquivo, nome_legenda, base_dados):
    """
    Lê um único arquivo CSV de modelo, calcula a produção total e retorna um
    DataFrame agregado e o comentário do modelo.

    Esta função foi otimizada para usar merges de Pandas em vez de iterrows.
    """
    try:
        logging.info(f"Processando arquivo: {arquivo} como '{nome_legenda}'")
        df_modelo = pd.read_csv(arquivo)

        # 1. Extrair e formatar comentário
        comentario = "Nenhum comentário disponível."
        if 'comentario' in df_modelo.columns and not df_modelo['comentario'].dropna().empty:
            comentario = df_modelo['comentario'].dropna().iloc[0]
        comentario_formatado = _format_comment_with_breaks(comentario)

        # 2. Preparar dados do modelo
        df_modelo['ds'] = pd.to_datetime(df_modelo['ds']) # converte a data para o formato datetime
        df_modelo['y_pred'] = pd.to_numeric(df_modelo['y_pred'], errors='coerce') # Converte o valores preditos para numerico, caso estejam em outro formato como string ou object
        df_teste = df_modelo[df_modelo['flag'] == "teste"].copy() # Filtra para ter apenas os dados do periodo de teste

        if df_teste.empty:
            logging.warning(f"Nenhum dado de 'teste' encontrado em {arquivo}.")
            return None, None, comentario_formatado

        df_teste['Ano'] = df_teste['ds'].dt.year # Converte a data para ter apenas o ano.
        df_teste.rename(columns={'unique_id': 'Municipio'}, inplace=True)

        # 3. Merge com base_dados para obter área colhida
        df_merged = pd.merge(
            df_teste[['treino_id', 'Municipio', 'Ano', 'y', 'y_pred']],
            base_dados,
            on=['Municipio', 'Ano'],
            how='left'
        )

        # 4. Calcular produção em sacas (evitando iteração)
        saca_kg = 60
        area_hectares = df_merged['Área colhida (Hectares)']

        # Coluna do modelo (Estimado)
        df_merged[nome_legenda] = (df_merged['y_pred'] * area_hectares * 1000) / saca_kg

        # Coluna Real (IBGE)
        nome_coluna_real = "Safra Real (IBGE)"
        df_merged[nome_coluna_real] = (df_merged['y'] * area_hectares * 1000) / saca_kg

        df_merged = df_merged.dropna(subset=[nome_legenda, nome_coluna_real, 'Área colhida (Hectares)'])

        # 5. Agregar por Ano
        df_agg = df_merged.groupby('Ano')[[nome_legenda, nome_coluna_real]].sum().reset_index()

        # Arredondar valores
        df_agg[nome_legenda] = df_agg[nome_legenda].round(2)
        df_agg[nome_coluna_real] = df_agg[nome_coluna_real].round(2)

        logging.info(f"Arquivo {arquivo} processado com sucesso.")

        # Retorna o DataFrame agregado e o DataFrame real separado
        df_estimado = df_agg[['Ano', nome_legenda]]
        df_real = df_agg[['Ano', nome_coluna_real]]

        return df_estimado, df_real, comentario_formatado

    except FileNotFoundError:
        logging.error(f"Erro: O arquivo {arquivo} não foi encontrado.")
        return None, None, None
    except Exception as e:
        logging.error(f"Ocorreu um erro ao processar o arquivo {arquivo}: {e}")
        return None, None, None

In [ ]:
def _preparar_dados_conab(df_conab_raw):
    """
    Formata o DataFrame da CONAB para o merge.
    """
    logging.info("Preparando os dados da CONAB.")
    nome_conab = "Estimativa CONAB"
    df_conab_est = df_conab_raw.T[['ESTIMADO']].rename(columns={'ESTIMADO': nome_conab})
    df_conab_est.index.name = 'Ano'
    df_conab_est.index = df_conab_est.index.astype(int)
    df_conab_est[nome_conab] *= 1000 # Convertendo de mil para unidades
    return df_conab_est, nome_conab

def _calcular_diferencas_percentuais(df, colunas_modelos, col_base='Safra Real (IBGE)'):
    """
    Calcula a diferença percentual de cada modelo em relação à coluna base (IBGE).
    """
    logging.info("Calculando as diferenças percentuais.")
    for col_estimada in colunas_modelos:
        if col_base in df and col_estimada in df:
            nome_diff = f"Diferença % ({col_estimada})"
            df[nome_diff] = ((df[col_estimada] - df[col_base]) / df[col_base]) * 100
        else:
            logging.warning(f"Coluna '{col_estimada}' ou '{col_base}' não encontrada para cálculo de diferença.")
    return df

In [ ]:
def processar_dados_para_csv(lista_arquivos_modelos, df_conab_raw, base_dados,
                           output_csv_path="dados_grafico_formatado.csv",
                           output_json_path="config_grafico.json"):
    """
    Orquestra a leitura, processamento, cálculo e unificação de todos os dados.
    Salva o resultado em um CSV formatado e um JSON de configuração.
    """
    logging.info("Iniciando o processamento e unificação dos dados...")

    lista_dfs_modelos = []
    lista_dfs_real = []
    ordem_modelos_estimados = []
    model_comments = {}

    # --- 1. Processar arquivos de modelo ---
    for arquivo, nome_legenda in lista_arquivos_modelos:
        df_estimado, df_real, comentario = _ler_e_processar_modelo_csv(arquivo, nome_legenda, base_dados)

        if df_estimado is not None and df_real is not None:
            lista_dfs_modelos.append(df_estimado)
            lista_dfs_real.append(df_real)
            ordem_modelos_estimados.append(nome_legenda)
            model_comments[nome_legenda] = comentario

    if not lista_dfs_modelos:
        logging.error("Nenhum arquivo de modelo foi processado com sucesso.")
        return

    # --- 2. Unificar dados Reais (IBGE) ---
    # Assume que os dados reais (y) são os mesmos em todos os arquivos
    df_real_unificado = pd.concat(lista_dfs_real).drop_duplicates().groupby('Ano').mean().reset_index()

    # --- 3. Unificar dados dos Modelos ---
    plot_df = reduce(lambda left, right: pd.merge(left, right, on='Ano', how='outer'), lista_dfs_modelos)
    plot_df = pd.merge(df_real_unificado, plot_df, on='Ano', how='left')

    # --- 4. Preparar e Unificar CONAB ---
    df_conab_est, nome_conab = _preparar_dados_conab(df_conab_raw)
    plot_df = pd.merge(plot_df, df_conab_est, on='Ano', how='left')
    plot_df = plot_df.sort_values('Ano').reset_index(drop=True)

    ordem_modelos_estimados.append(nome_conab)

    # --- 5. Adicionar comentários padrão ---
    ibge_comment = "Dados oficiais de safra do Instituto Brasileiro de Geografia e Estatística."
    conab_comment = "Estimativa oficial da Companhia Nacional de Abastecimento."
    model_comments['Safra Real (IBGE)'] = _format_comment_with_breaks(ibge_comment)
    model_comments[nome_conab] = _format_comment_with_breaks(conab_comment)

    # --- 6. Calcular Diferenças Percentuais ---
    plot_df = _calcular_diferencas_percentuais(plot_df, ordem_modelos_estimados)

    # --- 7. Salvar arquivos de saída ---
    try:
        plot_df.to_csv(output_csv_path, index=False, float_format='%.2f')
        logging.info(f"Dados formatados salvos com sucesso em: {output_csv_path}")

        config_data = {
            "comments": model_comments,
            "order": ordem_modelos_estimados,
            "conab_name": nome_conab,
            "ibge_name": "Safra Real (IBGE)"
        }
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(config_data, f, ensure_ascii=False, indent=4)
        logging.info(f"Configuração do gráfico salva com sucesso em: {output_json_path}")

    except Exception as e:
        logging.error(f"Erro ao salvar arquivos de saída: {e}")

In [ ]:
# --- Função de Geração de Gráfico (Parte 2) ---

def gerar_grafico_de_csv(input_csv_path, input_json_path, nome_arquivo_html="grafico_comparativo_cafe.html"):
    """
    Gera um gráfico comparativo usando Plotly a partir de um arquivo CSV
    formatado e um arquivo JSON de configuração.
    """

    # --- 1. Carregar dados processados ---
    logging.info("Iniciando a geração do gráfico a partir dos arquivos processados...")
    try:
        plot_df = pd.read_csv(input_csv_path)
        with open(input_json_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        model_comments = config.get('comments', {})
        ordem_modelos_estimados = config.get('order', [])
        nome_conab = config.get('conab_name', 'Estimativa CONAB')
        nome_ibge = config.get('ibge_name', 'Safra Real (IBGE)')

    except FileNotFoundError:
        logging.error(f"Erro: Arquivo CSV '{input_csv_path}' ou JSON '{input_json_path}' não encontrado.")
        return
    except Exception as e:
        logging.error(f"Erro ao ler os arquivos de entrada: {e}")
        return

    # --- 2. Gerando o Gráfico com Plotly ---
    logging.info("Gerando o gráfico com Plotly.")
    fig = go.Figure()

    # --- DEFINIÇÃO DE CORES FIXAS E CUSTOMIZADAS ---
    cor_ibge = 'darkblue'
    cor_conab = 'firebrick'
    cores_outros_modelos = [
      '#8EB38E', '#B3A7CC', '#E6B36C', '#B89877',
      '#D79FB5', '#9E9E9E', '#C6D07D', '#93C6CB',
      '#E6C48F', '#A77FBA',
      # Novas cores adicionadas:
      '#7FB1B3', '#D4A59A', '#A2C68D', '#C79FE2',
      '#E0A479', '#9AC1CF', '#B9D196', '#D9A8D0',
      '#8FC2C1', '#E6D29E'
    ]

    outros_modelos_estimados = [m for m in ordem_modelos_estimados if m != nome_conab]

    # --- BARRAS ---
    # 1. Barra IBGE
    fig.add_trace(go.Bar(
        x=plot_df['Ano'], y=plot_df[nome_ibge],
        name=nome_ibge, marker_color=cor_ibge,
        text=plot_df[nome_ibge].apply(lambda x: f'{x:,.0f}'), textposition='outside',
        customdata=[model_comments.get(nome_ibge)] * len(plot_df),
        hovertemplate='<b>Ano: %{x}</b><br><br><b>Fonte</b>: %{data.name}<br><b>Produção</b>: %{y:,.0f} sacas<br><br><b>Descrição</b>:<br>%{customdata}<extra></extra>'
    ))

    # 2. Barra CONAB
    fig.add_trace(go.Bar(
        x=plot_df['Ano'], y=plot_df[nome_conab],
        name=nome_conab, marker_color=cor_conab,
        text=plot_df[nome_conab].apply(lambda x: f'{x:,.0f}' if pd.notna(x) else ''), textposition='outside',
        customdata=[model_comments.get(nome_conab)] * len(plot_df),
        hovertemplate='<b>Ano: %{x}</b><br><br><b>Modelo</b>: %{data.name}<br><b>Produção</b>: %{y:,.0f} sacas<br><br><b>Descrição</b>:<br>%{customdata}<extra></extra>'
    ))

    # 3. Barras para outros modelos
    contador_cor = 0
    for col in outros_modelos_estimados:
        cor = cores_outros_modelos[contador_cor % len(cores_outros_modelos)]
        fig.add_trace(go.Bar(
            x=plot_df['Ano'], y=plot_df[col], name=col, marker_color=cor,
            text=plot_df[col].apply(lambda x: f'{x:,.0f}' if pd.notna(x) else ''), textposition='outside',
            customdata=[model_comments.get(col)] * len(plot_df),
            hovertemplate='<b>Ano: %{x}</b><br><br><b>Modelo</b>: %{data.name}<br><b>Produção</b>: %{y:,.0f} sacas<br><br><b>Descrição</b>:<br>%{customdata}<extra></extra>'
        ))
        contador_cor += 1

    # --- LINHAS DE DIFERENÇA PERCENTUAL ---
    # 1. Linha CONAB
    col_diff_conab = f"Diferença % ({nome_conab})"
    fig.add_trace(go.Scatter(
        x=plot_df['Ano'], y=plot_df[col_diff_conab], name=col_diff_conab,
        mode='lines+markers+text', yaxis='y2', line=dict(color=cor_conab, dash='dot'),
        text=plot_df[col_diff_conab].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else ''), textposition='top center', textfont=dict(size=10, color=cor_conab),
        customdata=[model_comments.get(nome_conab)] * len(plot_df),
        hovertemplate='<b>Ano: %{x}</b><br><br><b>Modelo</b>: ' + nome_conab + '<br><b>Diferença</b>: %{y:.2f}%<br><br><b>Descrição</b>:<br>%{customdata}<extra></extra>'
    ))

    # 2. Linhas para outros modelos
    contador_cor = 0
    for col_original in outros_modelos_estimados:
        cor = cores_outros_modelos[contador_cor % len(cores_outros_modelos)]
        col_diff = f"Diferença % ({col_original})"
        fig.add_trace(go.Scatter(
            x=plot_df['Ano'], y=plot_df[col_diff], name=col_diff,
            mode='lines+markers+text', yaxis='y2', line=dict(color=cor, dash='dot'),
            text=plot_df[col_diff].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else ''), textposition='top center', textfont=dict(size=10, color=cor),
            customdata=[model_comments.get(col_original)] * len(plot_df),
            hovertemplate='<b>Ano: %{x}</b><br><br><b>Modelo</b>: ' + col_original + '<br><b>Diferença</b>: %{y:.2f}%<br><br><b>Descrição</b>:<br>%{customdata}<extra></extra>'
        ))
        contador_cor += 1

    # --- 3. Layout do Gráfico e da Legenda ---
    fig.update_layout(
        title=f'<b>Produção de Café (Sacas): Modelos vs. Real (IBGE) e CONAB | {plot_df["Ano"].min()}-{plot_df["Ano"].max()}</b>',
        xaxis_title='<b>Ano</b>', yaxis_title='<b>Produção em Sacas</b>', barmode='group',
        template='plotly_white', height=700,
        legend=dict(
            title=dict(text="<b>Legenda</b><br>"),
            orientation="h",
            yanchor="bottom",
            y=-0.8,
            xanchor="center",
            x=0.5
        ),
        margin=dict(t=100, r=50, l=130, b=300), # Margem inferior aumentada para a legenda
        yaxis2=dict(
            title='<b>Diferença Percentual (%)</b>', overlaying='y', side='right',
            showgrid=False, zeroline=True, zerolinecolor='lightgrey', zerolinewidth=1
        )
    )

    # --- 4. Salvando e Exibindo o Gráfico ---
    try:
        logging.info(f"Salvando o gráfico como '{nome_arquivo_html}'...")
        fig.write_html(nome_arquivo_html)
        logging.info("Gráfico salvo com sucesso.")
    except Exception as e:
        logging.error(f"Ocorreu um erro ao salvar o gráfico em HTML: {e}")

    logging.info("Exibindo o gráfico...")
    fig.show()

## Grafico

In [9]:
if __name__ == "__main__":
    data_conab = {
        '2020': [31074.55, 34337.30442, 9.502069176], '2021': [20661.3, 21858.9, 5.478775236],
        '2022': [26687.4, 21570.1, 23.72404393], '2023': [27101.9, 28650.4, 5.404811102],
        '2024': [29836.4, 27708.3, 7.680370142], '2025': [24703.9, 24703.9, 0.0],
        '2026': [0.0, 0.0, 0.0]
    }
    df_conab_raw = pd.DataFrame(data_conab, index=['ESTIMADO', 'REAL', 'Erro'])

    try:
        base_dados_raw = pd.read_csv("Area_colhida_V2.csv")
        base_dados_raw.rename(columns={'Area': 'Área colhida (Hectares)'}, inplace=True)
        base_dados_raw = base_dados_raw[["Municipio", "Ano", "Área colhida (Hectares)"]]
    except FileNotFoundError:
        logging.error("ERRO: Arquivo 'Area_colhida_V2.csv' não encontrado. Abortando.")
        base_dados_raw = pd.DataFrame()

    # Modifique esta lista para alterar os arquivos, nomes e a ordem no gráfico
    lista_arquivos_input = [
      ("26_11_2025_features_por_cluster.csv", "Feature por cluster (27/11/2025)"),
      ("01_12_2025_teste_2016_2025_por_cluster.csv", "Feature por cluster 2016-2025"),
      ("modelo_altitude_ajustado.csv", "Modelo Altitude"),
    ]

    # Nomes dos arquivos intermediários
    formatted_date = current_date.strftime("%d_%m_%Y")
    PATH_CSV_FORMATADO = "dados_grafico_formatado.csv"
    PATH_JSON_CONFIG = "config_grafico.json"
    PATH_HTML_OUTPUT = f"grafico_sacas_cafe_{formatted_date}.html"

    # --- Bloco 2: Execução ---

    if not base_dados_raw.empty and lista_arquivos_input:
        # 1. Processa os dados e salva em arquivos CSV e JSON
        processar_dados_para_csv(
            lista_arquivos_modelos=lista_arquivos_input,
            df_conab_raw=df_conab_raw,
            base_dados=base_dados_raw,
            output_csv_path=PATH_CSV_FORMATADO,
            output_json_path=PATH_JSON_CONFIG
        )

        # 2. Gera o gráfico lendo os arquivos criados
        gerar_grafico_de_csv(
            input_csv_path=PATH_CSV_FORMATADO,
            input_json_path=PATH_JSON_CONFIG,
            nome_arquivo_html=PATH_HTML_OUTPUT
        )
    else:
        logging.error("Não foi possível executar o script devido à falta do 'Area_colhida_V2.csv' ou lista de arquivos vazia.")

In [ ]:
from datetime import datetime

mensagem = """
Esse grafico foi gerado para a reunião do dia 04/12/2025, com base nos pedidos realizados na reuinião de 27/11/2025.
Foi mostrado o resultado para 3 modelos diferentes:
- 01_12_2025_teste_2016_2025_por_cluster.csv onde cada cluster tem um conjunto específico de features.
  para cada ano de teste entre 2016 e 2025 foi treinado um modelo.
  os modelos estão salvos no diretorio "Treinos/01_12_2025_teste_2016_2025_por_cluster/Modelos".

- 26_11_2025_features_por_cluster.csv onde cada cluster tem um conjunto específico de features.
  para cada ano de teste entre 2020 e 2025 foi treinado um modelo.
  os modelos estão salvos no diretorio "Treinos/26_11_2025_features_por_cluster/Modelos".

- modelo_altitude_ajustado.csv neste modelo tem o uso da variavel de altitude.
  os modelos estão salvos no diretorio "Treinos/Modelo_Altitude_Ajustado/Modelos".

Os valores para a área destinada a colheita foi usado o dataset: Area_colhida_V2.csv
Todos os modelos foram treinados usando o mesmo conjunto de hiperparametros.
"""

# Obtém a data atual no formato dia_mês_ano
data_atual = datetime.now().strftime("%d_%m_%Y")

# Monta o nome do arquivo
nome_arquivo = f"grafico_{data_atual}.txt"

# Salva a string no arquivo, mantendo a formatação
with open(nome_arquivo, "w", encoding="utf-8") as arquivo:
    arquivo.write(mensagem)

print(f"Arquivo salvo como: {nome_arquivo}")

Arquivo salvo como: grafico_04_12_2025.txt


# Grafico da produtividade

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from functools import reduce
import logging
import textwrap

# Configuração do sistema de logs para monitorar a execução
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def gerar_grafico_produtividade_final(lista_arquivos_modelos, df_conab, base_dados_area, nome_arquivo_html="grafico_produtividade_teste.html"):
    """
    Gera um gráfico comparativo de produtividade (Ton/ha) focando apenas nos dados de TESTE.

    Lógica de Cálculo:
    1. IBGE e Modelos: Utiliza Média Ponderada pela Área (Município a Município).
       Fórmula: Soma(Produtividade_Município * Área_Município) / Soma(Área_Total)
    2. CONAB: Utiliza a produção total estimada dividida pela área total do arquivo CSV.

    Parâmetros:
    - lista_arquivos_modelos: Lista de tuplas (caminho_arquivo_csv, nome_legenda).
    - df_conab: DataFrame com dados de sacas da CONAB.
    - base_dados_area: DataFrame carregado do 'Area_colhida_V2.csv' contendo as áreas.
    """

    # Função auxiliar para quebrar linhas em textos de tooltip muito longos
    def format_comment_with_breaks(text, line_length=77):
        if not isinstance(text, str):
            return "Nenhum comentário disponível."
        return '<br>'.join(textwrap.wrap(text, width=line_length))

    if not lista_arquivos_modelos:
        logging.error("A lista de arquivos de modelos está vazia.")
        return

    logging.info("Iniciando o processamento dos modelos...")

    lista_dfs_finais = [] # Armazena os DataFrames processados de cada modelo
    ordem_modelos = []    # Mantém a ordem de plotagem
    model_comments = {}   # Armazena os comentários/descrições para o tooltip

    # ==============================================================================
    # 1. PROCESSAMENTO DOS MODELOS (CÁLCULO DA MÉDIA PONDERADA)
    # ==============================================================================
    for arquivo, nome_legenda in lista_arquivos_modelos:
        try:
            logging.info(f"Processando arquivo: {arquivo} ({nome_legenda})")
            df_modelo = pd.read_csv(arquivo)

            # Extração do comentário do modelo (se existir)
            comentario = "Nenhum comentário disponível."
            if 'comentario' in df_modelo.columns and not df_modelo['comentario'].dropna().empty:
                comentario = df_modelo['comentario'].dropna().iloc[0]
            model_comments[nome_legenda] = format_comment_with_breaks(comentario)

            # Conversão de tipos de dados
            df_modelo['ds'] = pd.to_datetime(df_modelo['ds'])
            df_modelo['y_pred'] = pd.to_numeric(df_modelo['y_pred'], errors='coerce')

            # --- FILTRO IMPORTANTE: Apenas dados de 'teste' ---
            # Aqui garantimos que apenas os anos marcados como teste (ex: 2025) sejam processados.
            cols_necessarias = ["unique_id", "ds", "y", "y_pred"]
            df_processar = df_modelo[df_modelo['flag'] == "teste"][cols_necessarias].copy()

            if df_processar.empty:
                logging.warning(f"O arquivo {arquivo} não possui dados com flag='teste'. Ignorando.")
                continue

            # Lista temporária para armazenar o cálculo de volume (Ton) por município
            temp_data = []

            for _, row in df_processar.iterrows():
                ano = row["ds"].year
                municipio_id = row["unique_id"]

                # Busca a área correspondente no arquivo único de áreas (Area_colhida_V2)
                filtro_area = (base_dados_area['Municipio'] == municipio_id) & (base_dados_area['Ano'] == ano)
                area_encontrada = base_dados_area.loc[filtro_area, 'Area_Hectares']

                if not area_encontrada.empty:
                    area = area_encontrada.values[0]

                    # CÁLCULO DE VOLUME (TONELADAS):
                    # Multiplicamos a produtividade (Ton/ha) pela área (ha) para obter o peso correto do município
                    volume_real_ton = row["y"] * area
                    volume_pred_ton = row["y_pred"] * area

                    temp_data.append({
                        "Ano": ano,
                        "Area": area,
                        "Vol_Real_Ton": volume_real_ton,
                        "Vol_Pred_Ton": volume_pred_ton
                    })

            # Se nenhum município foi cruzado corretamente, pula para o próximo modelo
            if not temp_data:
                logging.warning(f"Nenhum dado de área encontrado para cruzar com {nome_legenda}")
                continue

            # Criação do DataFrame intermediário
            df_calc = pd.DataFrame(temp_data)

            # Agrupa por ANO somando Volumes e Áreas (Consolidação Estadual/Nacional)
            df_agg = df_calc.groupby('Ano')[['Area', 'Vol_Real_Ton', 'Vol_Pred_Ton']].sum().reset_index()

            # CÁLCULO FINAL DA PRODUTIVIDADE MÉDIA PONDERADA:
            # Produtividade = Volume Total (Ton) / Área Total (ha)
            nome_col_real = "Produtividade Real (IBGE)"

            # Uso de lambda para evitar erro de divisão por zero
            df_agg[nome_col_real] = df_agg.apply(lambda x: x['Vol_Real_Ton'] / x['Area'] if x['Area'] > 0 else 0, axis=1).round(2)
            df_agg[nome_legenda] = df_agg.apply(lambda x: x['Vol_Pred_Ton'] / x['Area'] if x['Area'] > 0 else 0, axis=1).round(2)

            # Tratamento visual: Se o Real for zero (comum em anos futuros de teste), transforma em NaN para não plotar barra vazia
            # Isso assume que se é 'teste', provavelmente não temos o Real oficial fechado ainda
            df_agg.loc[df_agg[nome_col_real] == 0, nome_col_real] = np.nan

            # Adiciona comentário padrão para a coluna do IBGE se ainda não existir
            if nome_col_real not in model_comments:
                model_comments[nome_col_real] = format_comment_with_breaks("Produtividade calculada via média ponderada utilizando a área do arquivo 'Area_colhida_V2'.")

            # Armazena o resultado final (apenas colunas necessárias)
            lista_dfs_finais.append(df_agg[['Ano', nome_col_real, nome_legenda]])
            ordem_modelos.append(nome_legenda)

        except Exception as e:
            logging.error(f"Erro crítico ao processar {arquivo}: {e}")
            continue

    if not lista_dfs_finais:
        logging.error("Nenhum modelo foi processado com sucesso. Abortando.")
        return

    # União (Merge) de todos os DataFrames gerados em um único DataFrame mestre para plotagem
    # Usa o 'Ano' e 'Produtividade Real' como chaves para alinhar tudo
    plot_df = reduce(lambda left, right: pd.merge(left, right, on=['Ano', 'Produtividade Real (IBGE)'], how='outer'), lista_dfs_finais)
    plot_df = plot_df.sort_values('Ano').reset_index(drop=True)

    # ==============================================================================
    # 2. CÁLCULO DA PRODUTIVIDADE CONAB
    # ==============================================================================
    logging.info("Calculando produtividade estimada CONAB...")
    nome_conab = "Produtividade CONAB (Est.)"

    # Calcula a área total por ano usando o MESMO arquivo CSV usado nos modelos
    df_area_total_ano = base_dados_area.groupby('Ano')['Area_Hectares'].sum().reset_index()

    # Prepara os dados de produção da CONAB (Originalmente em Sacas)
    df_conab_sacas = df_conab.T[['ESTIMADO']].rename(columns={'ESTIMADO': 'Sacas_Conab'})
    df_conab_sacas.index.name = 'Ano'
    df_conab_sacas.index = df_conab_sacas.index.astype(int)
    df_conab_sacas['Sacas_Conab'] = df_conab_sacas['Sacas_Conab'] * 1000 # Ajuste para unidade (milhares de sacas -> sacas)

    # Cruza Produção Conab com Área Total IBGE (CSV)
    df_conab_calc = pd.merge(df_conab_sacas, df_area_total_ano, on='Ano', how='inner')

    # Fórmula CONAB:
    # 1. Sacas * 60 = Kg
    # 2. Kg / 1000 = Toneladas
    # 3. Toneladas / Hectares = Ton/ha
    df_conab_calc[nome_conab] = ((df_conab_calc['Sacas_Conab'] * 60) / 1000) / df_conab_calc['Area_Hectares']
    df_conab_calc[nome_conab] = df_conab_calc[nome_conab].round(2)

    # Adiciona coluna CONAB ao DataFrame principal
    plot_df = pd.merge(plot_df, df_conab_calc[['Ano', nome_conab]], on='Ano', how='left')
    ordem_modelos.append(nome_conab)

    model_comments[nome_conab] = format_comment_with_breaks("Produtividade derivada: (Produção CONAB / Área Total do CSV Area_colhida_V2).")

    # ==============================================================================
    # 3. CÁLCULO DAS DIFERENÇAS PERCENTUAIS
    # ==============================================================================
    col_real = "Produtividade Real (IBGE)"

    for col in ordem_modelos:
        if col == col_real: continue
        if col in plot_df.columns and col_real in plot_df.columns:
            nome_diff = f"Diferença % ({col})"
            # A diferença só é calculada se houver um valor Real válido (o que pode não ocorrer no futuro/teste)
            plot_df[nome_diff] = np.where(
                (plot_df[col_real] > 0) & (plot_df[col_real].notna()),
                ((plot_df[col] - plot_df[col_real]) / plot_df[col_real]) * 100,
                np.nan
            )

    # ==============================================================================
    # 4. GERAÇÃO DO GRÁFICO COM PLOTLY
    # ==============================================================================
    logging.info("Gerando visualização Plotly...")
    fig = go.Figure()

    # Definição de paleta de cores
    cor_ibge = 'darkblue'
    cor_conab = 'firebrick'
    cores_outros = ['#8EB38E', '#B3A7CC', '#E6B36C', '#B89877', '#D79FB5', '#93C6CB']

    # --- BARRA: Real (IBGE) ---
    if col_real in plot_df.columns:
        df_plot_real = plot_df[plot_df[col_real] > 0] # Filtra valores vazios
        fig.add_trace(go.Bar(
            x=df_plot_real['Ano'], y=df_plot_real[col_real],
            name=col_real, marker_color=cor_ibge,
            text=df_plot_real[col_real], textposition='outside',
            customdata=[model_comments.get(col_real)] * len(df_plot_real),
            hovertemplate='<b>Ano: %{x}</b><br>Fonte: IBGE<br>Valor: %{y} Ton/ha<br><br>%{customdata}<extra></extra>'
        ))

    # --- BARRA: CONAB ---
    if nome_conab in plot_df.columns:
        # Filtra para exibir apenas anos relevantes que estão no plot_df (anos de teste)
        df_c = plot_df[plot_df[nome_conab].notna() & (plot_df[nome_conab] > 0)]
        fig.add_trace(go.Bar(
            x=df_c['Ano'], y=df_c[nome_conab],
            name=nome_conab, marker_color=cor_conab,
            text=df_c[nome_conab], textposition='outside',
            customdata=[model_comments.get(nome_conab)] * len(df_c),
            hovertemplate='<b>Ano: %{x}</b><br>Fonte: CONAB<br>Valor: %{y} Ton/ha<br><br>%{customdata}<extra></extra>'
        ))

    # --- BARRAS: Outros Modelos ---
    idx_cor = 0
    modelos_extras = [m for m in ordem_modelos if m != nome_conab and m != col_real]

    for col in modelos_extras:
        if col in plot_df.columns:
            cor = cores_outros[idx_cor % len(cores_outros)]
            df_m = plot_df[plot_df[col] > 0] # Filtra zeros
            fig.add_trace(go.Bar(
                x=df_m['Ano'], y=df_m[col],
                name=col, marker_color=cor,
                text=df_m[col], textposition='outside',
                customdata=[model_comments.get(col)] * len(df_m),
                hovertemplate=f'<b>Ano: %{{x}}</b><br>Modelo: {col}<br>Valor: %{{y}} Ton/ha<br><br>%{{customdata}}<extra></extra>'
            ))
            idx_cor += 1

    # --- LINHAS: Diferença Percentual ---
    for col in ordem_modelos:
        if col == col_real: continue
        col_diff = f"Diferença % ({col})"

        if col_diff in plot_df.columns:
            df_l = plot_df.dropna(subset=[col_diff])
            if df_l.empty: continue # Se não houver dados de diferença (ex: falta o Real), não plota linha

            # Define a cor da linha combinando com a barra correspondente
            cor_linha = cor_conab if col == nome_conab else cores_outros[ordem_modelos.index(col) % len(cores_outros)]
            if col == nome_conab: cor_linha = cor_conab

            fig.add_trace(go.Scatter(
                x=df_l['Ano'], y=df_l[col_diff],
                name=f"Diferença % ({col})",
                mode='lines+markers+text', yaxis='y2', line=dict(color=cor_linha, dash='dot'),
                text=df_l[col_diff].apply(lambda x: f'{x:.1f}%'),
                textposition='top center', textfont=dict(size=10)
            ))

    # --- LAYOUT E SALVAMENTO ---
    fig.update_layout(
        title='<b>Produtividade de Café (Ton/ha) - Período de Teste</b>',
        xaxis=dict(title='<b>Ano</b>', type='category'), # 'category' força mostrar apenas os anos existentes sem decimais
        yaxis_title='<b>Produtividade (Ton/ha)</b>',
        barmode='group',
        template='plotly_white', height=700,
        legend=dict(orientation="h", yanchor="bottom", y=-0.6, xanchor="center", x=0.5),
        margin=dict(b=250),
        yaxis2=dict(title='<b>Diferença (%)</b>', overlaying='y', side='right', showgrid=False)
    )

    logging.info(f"Salvando gráfico em: {nome_arquivo_html}")
    try:
        fig.write_html(nome_arquivo_html)
        fig.show()
        logging.info("Processo concluído com sucesso.")
    except Exception as e:
        logging.error(f"Erro ao salvar/exibir o gráfico: {e}")


# ==============================================================================
# EXECUÇÃO DO SCRIPT
# ==============================================================================

# 1. Definição dos dados brutos da CONAB (Sacas)
data_conab = {
    '2020': [31074.55, 34337.30442, 9.502069176], '2021': [20661.3, 21858.9, 5.478775236],
    '2022': [26687.4, 21570.1, 23.72404393], '2023': [27101.9, 28650.4, 5.404811102],
    '2024': [29836.4, 27708.3, 7.680370142], '2025': [24703.9, 24703.9, 0.0],
    '2026': [0.0, 0.0, 0.0]
}
df_conab = pd.DataFrame(data_conab, index=['ESTIMADO', 'REAL', 'Erro'])

# 2. Carregamento e preparação do arquivo de Áreas (FONTE ÚNICA)
try:
    logging.info("Carregando base de dados de áreas (Area_colhida_V2.csv)...")
    base_dados = pd.read_csv("Area_colhida_V2.csv")

    # Padronização dos nomes das colunas para evitar erros (Area vs Área colhida...)
    if 'Area' in base_dados.columns:
        base_dados.rename(columns={'Area': 'Area_Hectares'}, inplace=True)
    elif 'Área colhida (Hectares)' in base_dados.columns:
        base_dados.rename(columns={'Área colhida (Hectares)': 'Area_Hectares'}, inplace=True)

    # Mantém apenas colunas essenciais
    base_dados = base_dados[["Municipio", "Ano", "Area_Hectares"]]

    # Definição da lista de modelos a serem processados
    # Formato: (Caminho do CSV, Nome para Legenda)
    lista_arquivos = [
      ("modelo_altitude.csv", "Estimativa SEBRAE - Altitude"),
      ("modelo_altitude_ajustado.csv", "Estimativa SEBRAE - Altitude Ajustado"),
      ("modelo_altitude_ajustado.csv", "Estimativa SEBRAE - Altitude Ajustado"),
      # Adicione outros modelos aqui...
    ]

    # Executa a geração do gráfico se a base de dados for válida
    if not base_dados.empty:
        gerar_grafico_produtividade_final(lista_arquivos, df_conab, base_dados)

except FileNotFoundError:
    logging.error("ERRO CRÍTICO: Arquivo 'Area_colhida_V2.csv' não encontrado. Verifique o diretório.")
except Exception as e:
    logging.error(f"Erro não esperado na preparação dos dados: {e}")

In [ ]:
import pandas as pd
import plotly.graph_objects as go



# Dados de 2020 - 2026
data = {
    'Safra Real IBGE': [33996367, 22340050, 22818800, 28483900, 27641233, 0, 0],
    'Estimativa CONAB': [31074550, 20661300, 26687400, 27101900, 29836400, 24703900, 0],
    'Estimativa SEBRAE': [31329453, 27971152, 21549294, 27672381, 30735952, 25470456, 0],
    'Estimativa SEBRAE - Predições Meteorologicas': [33911478, 27052299, 34348700, 18978972, 23485670, 25745075, 25626520] # pandemia_base_Hmax
}

# Anos correspondentes aos dados.
index = [2020, 2021, 2022, 2023, 2024, 2025, 2026]

colors = {
    'Safra Real IBGE': 'darkblue',
    'Estimativa CONAB': 'red',
    'Estimativa SEBRAE': 'green',
    #'Estimativa SEBRAE - Predições Meteorologicas': 'yellow'
}



# --- Definição dos Dados ---
# Os dados fornecidos são colocados num dicionário.
data = {
    'IBGE': [28483900, 27641233, 0, 0],
    'CONAB': [27101900, 29836400, 24703900, 0],
    'SEBRAE': [27672381, 30735952, 25745075, 25626520],
}

# Anos correspondentes aos dados.
index = [2023, 2024, 2025, 2026]

# Criação de um DataFrame do Pandas para facilitar a manipulação.
df = pd.DataFrame(data, index=index)

# --- Criação do Gráfico ---
# Inicializa a figura do gráfico.
fig = go.Figure()

# Define as cores para cada estimativa, conforme solicitado.
colors = {
    'IBGE': 'darkblue',
    'CONAB': 'red',
    'SEBRAE': 'green',
}

# Adiciona uma barra ao gráfico para cada coluna de dados.
for col in df.columns:
    fig.add_trace(go.Bar(
        x=df.index,
        y=df[col],
        name=col,
        marker_color=colors[col],
        text=df[col].apply(lambda x: f'{x:,.0f}'.replace(',', '.')), # Formata o texto para exibição
        textposition='auto',
    ))

# --- Personalização do Layout ---
# Atualiza o layout do gráfico para corresponder ao design de referência.
fig.update_layout(
    title_text='<b>Comparativo de Estimativas de Safra</b>',
    xaxis_title='Ano',
    yaxis_title='Produção em Sacas',
    barmode='group',  # Agrupa as barras por ano.
    plot_bgcolor='white',  # Fundo branco.
    xaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='lightgray'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgray'
    ),
    legend=dict(
        orientation="h",  # Legenda horizontal.
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="black"
    )
)

# --- Exibição e Exportação ---
# Mostra o gráfico interativo.
fig.show()

# Opcional: Salva o gráfico como um ficheiro HTML interativo.
# fig.write_html('comparativo_estimativas_safra.html')